In [2]:
import cv2
import os
from ultralytics import YOLO
import time # Para medir el tiempo de procesamiento

# Nota: glob y matplotlib ya no son necesarios para la lógica principal del video.

def dibujar_resultados_y_verificar(frame, resultados, modelo, color=(255, 0, 255)):
    """
    Dibuja las cajas de detección en un frame y devuelve los nombres de las clases detectadas.
    """
    clases_detectadas = set() # Usamos un set para evitar duplicados

    for r in resultados:
        for box in r.boxes:
            # Extraer información de la caja
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls_id = int(box.cls[0])
            nombre_clase = modelo.names[cls_id]

            # Añadir la clase al set
            clases_detectadas.add(nombre_clase)

            # Dibujar el rectángulo
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            # Preparar y dibujar la etiqueta
            etiqueta = f"{nombre_clase} {conf:.2f}"
            cv2.putText(frame, etiqueta, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    return frame, clases_detectadas

In [12]:
# --- RUTAS ---
# Rutas a tus modelos .pt
ruta_modelo_accidentes = 'best_accident.pt'
ruta_modelo_fuego = 'best_fire.pt'
ruta_modelo_personas = 'best_general.pt'

# Ruta del video que quieres analizar
ruta_video_entrada = 'videos/video8.mp4' # <-- CAMBIA ESTO por la ruta de tu video

# Ruta donde se guardará el video con las detecciones
ruta_video_salida = 'videos/salida/video_salida8.mp4'

# --- PARÁMETROS DE DETECCIÓN ---
# Umbrales de confianza para cada modelo
confianza_accidentes = 0.85
confianza_fuego = 0.4
confianza_personas = 0.5

# --- PARÁMETRO DE RENDIMIENTO ---
# Analizar un fotograma cada 'x' fotogramas.
# 1 = analizar todos. 5 = analizar 1 de cada 5. Mayor número = más rápido.
procesar_cada_x_frames = 30

# Cargar modelo de accidentes
try:
    modelo_accidentes = YOLO(ruta_modelo_accidentes)
    print("✅ Modelo de ACCIDENTES cargado exitosamente.")
    print("   Clases:", modelo_accidentes.names)
except Exception as e:
    print(f"❌ Error al cargar el modelo de accidentes '{ruta_modelo_accidentes}': {e}")
    modelo_accidentes = None

# Cargar modelo de fuego y humo
try:
    modelo_fuego = YOLO(ruta_modelo_fuego)
    print("✅ Modelo de FUEGO/HUMO cargado exitosamente.")
    print("   Clases:", modelo_fuego.names)
except Exception as e:
    print(f"❌ Error al cargar el modelo de fuego '{ruta_modelo_fuego}': {e}")
    modelo_fuego = None



# Cargar modelo de personas
try:
    modelo_personas = YOLO(ruta_modelo_personas)
    print("✅ Modelo de personas cargado exitosamente.")
    print("   Clases:", modelo_personas.names)
except Exception as e:
    print(f"❌ Error al cargar el modelo de personas: {e}")
    modelo_personas = None

✅ Modelo de ACCIDENTES cargado exitosamente.
   Clases: {0: 'accident', 1: 'moderate', 2: 'severe'}
✅ Modelo de FUEGO/HUMO cargado exitosamente.
   Clases: {0: 'Fire', 1: 'Smoke'}
✅ Modelo de personas cargado exitosamente.
   Clases: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'bus', 5: 'train', 6: 'truck', 7: 'cat', 8: 'dog', 9: 'backpack', 10: 'handbag', 11: 'suitcase', 12: 'sports ball', 13: 'baseball bat', 14: 'skateboard', 15: 'tennis racket', 16: 'bottle', 17: 'fork', 18: 'knife', 19: 'spoon', 20: 'scissors', 21: 'handgun', 22: 'Fire'}


In [13]:
# <-- MODIFICADO: Verificar que los TRES modelos y el video de entrada existan -->
if modelo_accidentes is None or modelo_fuego is None or modelo_personas is None:
    print("⚠️ No se puede continuar porque uno o más modelos no se cargaron correctamente.")
elif not os.path.exists(ruta_video_entrada):
    print(f"❌ Error: El video de entrada no se encontró en '{ruta_video_entrada}'")
else:
    # --- INICIALIZACIÓN DEL VIDEO ---
    cap = cv2.VideoCapture(ruta_video_entrada)
    if not cap.isOpened():
        print(f"❌ Error al abrir el archivo de video: {ruta_video_entrada}")
    else:
        # Obtener propiedades del video para el archivo de salida
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))

        # Definir el codec y crear el objeto VideoWriter
        fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec para .mp4
        out = cv2.VideoWriter(ruta_video_salida, fourcc, fps, (frame_width, frame_height))

        print(f"\n🚀 Iniciando procesamiento del video: {ruta_video_entrada}")
        print(f"   - Analizando 1 de cada {procesar_cada_x_frames} fotogramas.")
        print(f"   - El video de salida se guardará en: {ruta_video_salida}")

        # --- VARIABLES PARA EL BUCLE ---
        frame_count = 0
        start_time = time.time()
        
        # <-- MODIFICADO: Banderas de alerta para el resumen final -->
        alerta_fuego_activada = False
        alerta_accidente_activada = False
        alerta_personas_activada = False # <-- NUEVO

        # <-- MODIFICADO: Almacenar los últimos resultados para dibujarlos en frames intermedios -->
        ultimos_resultados_accidentes = []
        ultimos_resultados_fuego = []
        ultimos_resultados_personas = [] # <-- NUEVO

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break # Fin del video

            frame_count += 1
            
            # --- LÓGICA DE PROCESAMIENTO CADA X FRAMES ---
            if frame_count % procesar_cada_x_frames == 0:
                # Solo en estos fotogramas ejecutamos los modelos
                print(f"   Analizando fotograma #{frame_count}...")
                
                # Ejecutar modelo de ACCIDENTES
                ultimos_resultados_accidentes = modelo_accidentes.predict(source=frame, conf=confianza_accidentes, verbose=False)
                
                # Ejecutar modelo de FUEGO/HUMO
                ultimos_resultados_fuego = modelo_fuego.predict(source=frame, conf=confianza_fuego, verbose=False)

                # <-- NUEVO: Ejecutar modelo de personas -->
                ultimos_resultados_personas = modelo_personas.predict(source=frame, conf=confianza_personas, verbose=False)

            # --- DIBUJAR Y VERIFICAR ALERTAS (en todos los frames) ---
            # Usamos los últimos resultados guardados para que el video sea fluido
            clases_detectadas_total = set()
            
            if ultimos_resultados_accidentes:
                frame, clases_acc = dibujar_resultados_y_verificar(frame, ultimos_resultados_accidentes, modelo_accidentes, color=(0, 255, 255)) # Cyan
                clases_detectadas_total.update(clases_acc)
            
            if ultimos_resultados_fuego:
                frame, clases_fue = dibujar_resultados_y_verificar(frame, ultimos_resultados_fuego, modelo_fuego, color=(0, 0, 255)) # Rojo
                clases_detectadas_total.update(clases_fue)
            
            # <-- NUEVO: Dibujar resultados del modelo de personas -->
            if ultimos_resultados_personas:
                # Usamos un color diferente, por ejemplo, magenta
                frame, clases_arm = dibujar_resultados_y_verificar(frame, ultimos_resultados_personas, modelo_personas, color=(255, 0, 255)) # Magenta
                clases_detectadas_total.update(clases_arm)


            # <-- MODIFICADO: Comprobar si se debe activar alguna alerta global -->
            # (Asume nombres de clases. ¡Ajústalos si tus modelos usan otros nombres!)
            if any(cls in ['Fire', 'Smoke'] for cls in clases_detectadas_total):
                alerta_fuego_activada = True
            
            if any(cls in ['Accident', 'accident', 'accidente'] for cls in clases_detectadas_total):
                alerta_accidente_activada = True

            # <-- NUEVO: Comprobar alerta de personas -->
            if any(cls in ['Weapon', 'weapon', 'gun', 'pistol', 'rifle', 'arma', 'pistola'] for cls in clases_detectadas_total):
                alerta_personas_activada = True

            # Escribir el frame procesado en el video de salida
            out.write(frame)

        # --- FINALIZACIÓN ---
        end_time = time.time()
        total_time = end_time - start_time
        
        # Liberar recursos
        cap.release()
        out.release()
        cv2.destroyAllWindows()

        print("\n✅ ¡Proceso de video completado!")
        print(f"   - Tiempo total de procesamiento: {total_time:.2f} segundos.")
        print(f"   - Total de fotogramas procesados: {frame_count}")

        # <-- MODIFICADO: INFORME FINAL DE ALERTAS ---
        print("\n--- 🚨 INFORME DE ALERTAS DEL VIDEO 🚨 ---")
        if not alerta_fuego_activada and not alerta_accidente_activada and not alerta_personas_activada:
            print("🟢 No se detectaron riesgos de incendio, accidentes o personas en el video.")
        else:
            if alerta_fuego_activada:
                print("🔥 ALERTA: Se ha detectado RIESGO DE INCENDIO/HUMO en el video.")
            if alerta_accidente_activada:
                print("🚗 ALERTA: Se ha detectado un posible ACCIDENTE AUTOMOVILÍSTICO en el video.")
            if alerta_personas_activada:
                print("🔫 ALERTA: Se ha detectado la presencia de personas en el video.") # <-- NUEVO
        print("------------------------------------------")


🚀 Iniciando procesamiento del video: videos/video8.mp4
   - Analizando 1 de cada 30 fotogramas.
   - El video de salida se guardará en: videos/salida/video_salida8.mp4
   Analizando fotograma #30...
   Analizando fotograma #60...
   Analizando fotograma #90...
   Analizando fotograma #120...
   Analizando fotograma #150...
   Analizando fotograma #180...
   Analizando fotograma #210...
   Analizando fotograma #240...
   Analizando fotograma #270...
   Analizando fotograma #300...
   Analizando fotograma #330...
   Analizando fotograma #360...
   Analizando fotograma #390...
   Analizando fotograma #420...
   Analizando fotograma #450...
   Analizando fotograma #480...
   Analizando fotograma #510...
   Analizando fotograma #540...
   Analizando fotograma #570...
   Analizando fotograma #600...
   Analizando fotograma #630...
   Analizando fotograma #660...
   Analizando fotograma #690...
   Analizando fotograma #720...
   Analizando fotograma #750...
   Analizando fotograma #780...
  

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'


In [14]:
import os
import google.generativeai as genai

# Asegúrate de que tu API Key esté configurada como variable de entorno
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

print("Modelos disponibles para tu API Key:")
print("-----------------------------------")
for m in genai.list_models():
    # Solo mostramos los modelos que pueden generar contenido o embeddings,
    # ya que son los más comunes para la clase GenerativeModel y embed_content.
    if "generateContent" in m.supported_generation_methods or "embedContent" in m.supported_generation_methods:
        print(f"Nombre: {m.name}")
        print(f"Descripción: {m.description}")
        print(f"Métodos de generación soportados: {m.supported_generation_methods}")
        print(f"Límite de tokens de entrada: {m.input_token_limit}")
        print(f"Límite de tokens de salida: {m.output_token_limit}")
        print("-" * 30)

Modelos disponibles para tu API Key:
-----------------------------------
Nombre: models/gemini-1.0-pro-vision-latest
Descripción: The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.
Métodos de generación soportados: ['generateContent', 'countTokens']
Límite de tokens de entrada: 12288
Límite de tokens de salida: 4096
------------------------------
Nombre: models/gemini-pro-vision
Descripción: The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.
Métodos de generación soportados: ['generateContent', 'countTokens']
Límite de tokens de entrada: 12288
Límite de tokens de salida: 4096
------------------------------
Nombre: models/gemini-1.5-pro-latest
Descripción: Alias that points to the most recent production (non-experimental) release o